# Tải dữ liệu

In [ ]:
import pandas as pd
# Đọc file CSV
file_path = "fitness_plans_updated.csv"  # Thay "your_file.csv" bằng đường dẫn tới file của bạn
data = pd.read_csv(file_path)


# Kiểm tra thông tin cơ bản
print(data.head())
print(data.info())
print(data['plan_id'].value_counts())

   plan_id  gender    fitness_goal  age    bmi
0       12       0  MuscleBuilding   43  32.07
1       41       0          Cardio   41  26.48
2        5       1  MuscleBuilding   34  21.93
3       17       0          Cardio   50  16.14
4       48       1  MuscleBuilding   21  42.86
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   plan_id       1000 non-null   int64  
 1   gender        1000 non-null   int64  
 2   fitness_goal  1000 non-null   object 
 3   age           1000 non-null   int64  
 4   bmi           1000 non-null   float64
dtypes: float64(1), int64(3), object(1)
memory usage: 39.2+ KB
None
plan_id
20    30
26    30
36    28
30    28
43    28
14    26
44    26
24    25
48    25
1     23
8     23
25    22
19    22
42    22
37    22
41    22
7     22
17    21
9     21
2     21
13    21
16    20
40    20
3     20
4     20
34    20
31

# Tiền xử lý dữ liệu

In [64]:
from sklearn.preprocessing import LabelEncoder

# Mã hóa fitness_goal
label_encoder = LabelEncoder()
data['fitness_goal'] = label_encoder.fit_transform(data['fitness_goal'])

# Chia thành đầu vào X và đầu ra y
X = data.drop('plan_id', axis=1)
y = data['plan_id']


# Chia tập dữ liệu

In [65]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


# Tăng cường dữ liệu bằng SMOTE

In [66]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Kiểm tra lại phân phối dữ liệu sau khi tăng cường
print(y_train_resampled.value_counts())


plan_id
38    24
13    24
8     24
9     24
22    24
44    24
15    24
16    24
42    24
34    24
26    24
49    24
20    24
30    24
23    24
37    24
46    24
28    24
17    24
35    24
25    24
11    24
29    24
21    24
2     24
43    24
14    24
24    24
31    24
6     24
3     24
18    24
41    24
39    24
27    24
19    24
32    24
10    24
36    24
5     24
45    24
40    24
4     24
33    24
12    24
47    24
7     24
48    24
50    24
1     24
Name: count, dtype: int64


# Huấn luyện mô hình

In [67]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report

# Điều chỉnh nhãn lớp để bắt đầu từ 0
y_train_resampled_adjusted = y_train_resampled - y_train_resampled.min()
y_test_adjusted = y_test - y_test.min()

# Khởi tạo và huấn luyện mô hình
model = XGBClassifier(objective="multi:softmax", num_class=len(y.unique()), eval_metric="mlogloss")
model.fit(X_train_resampled, y_train_resampled_adjusted)

# Dự đoán trên tập kiểm tra
y_pred_adjusted = model.predict(X_test)

# Chuyển kết quả dự đoán về nhãn lớp ban đầu (nếu cần)
y_pred = y_pred_adjusted + y_test.min()

# Đánh giá mô hình
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")
print(classification_report(y_test, y_pred))


Accuracy: 0.03
              precision    recall  f1-score   support

           1       0.00      0.00      0.00         5
           2       0.25      0.25      0.25         4
           3       0.00      0.00      0.00         4
           4       0.00      0.00      0.00         4
           5       0.00      0.00      0.00         3
           6       0.00      0.00      0.00         3
           7       0.00      0.00      0.00         4
           8       0.00      0.00      0.00         5
           9       0.00      0.00      0.00         4
          10       0.00      0.00      0.00         3
          11       0.00      0.00      0.00         4
          12       0.00      0.00      0.00         2
          13       0.00      0.00      0.00         4
          14       0.00      0.00      0.00         5
          15       0.00      0.00      0.00         3
          16       0.40      0.50      0.44         4
          17       0.00      0.00      0.00         4
          18

d:\AppData\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\AppData\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\AppData\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


# Phân tích kết quả

In [68]:
from sklearn.model_selection import GridSearchCV

# Điều chỉnh nhãn lớp để bắt đầu từ 0
y_train_resampled_adjusted = y_train_resampled - y_train_resampled.min()

param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
}

# Sử dụng model đã huấn luyện với nhãn lớp đã điều chỉnh
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='accuracy')
grid.fit(X_train_resampled, y_train_resampled_adjusted)

# In kết quả tốt nhất
print(f"Best parameters: {grid.best_params_}")
best_model = grid.best_estimator_

# Nếu cần chuyển đổi lại nhãn lớp về dạng ban đầu
y_pred_adjusted = best_model.predict(X_test)
y_pred = y_pred_adjusted + y_test.min()

# Đánh giá mô hình
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")
print(classification_report(y_test, y_pred))
 

Best parameters: {'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 100}
Accuracy: 0.01
              precision    recall  f1-score   support

           1       0.00      0.00      0.00         5
           2       0.00      0.00      0.00         4
           3       0.00      0.00      0.00         4
           4       0.00      0.00      0.00         4
           5       0.00      0.00      0.00         3
           6       0.00      0.00      0.00         3
           7       0.00      0.00      0.00         4
           8       0.00      0.00      0.00         5
           9       0.00      0.00      0.00         4
          10       0.00      0.00      0.00         3
          11       0.00      0.00      0.00         4
          12       0.00      0.00      0.00         2
          13       0.00      0.00      0.00         4
          14       0.00      0.00      0.00         5
          15       0.00      0.00      0.00         3
          16       0.00      0.00      0.00

# Dự đoán dữ liệu mới

In [84]:
# Ví dụ dữ liệu mới
new_data = pd.DataFrame({
    'gender': [0],
    'fitness_goal': [1],  # Mã hóa fitness_goal tương tự như trên
    'age': [30],
    'bmi': [25.5]
})

# Dự đoán
new_prediction = model.predict(new_data)
print("Predicted plan_id:", new_prediction)


Predicted plan_id: [27]


# Lưu mô hình

In [ ]:
import pickle

# Lưu mô hình vào file với pickle
with open('xgb_model.pkl', 'wb') as file:
    pickle.dump(model, file)


# Tải lại mô hình với pickle
# with open('xgb_model.pkl', 'rb') as file:
#     loaded_model = pickle.load(file)

# Dự đoán với mô hình đã tải
# predictions = loaded_model.predict(new_data)
# print("Predicted plan_id:", predictions)

Predicted plan_id: [27]
